<a href="https://colab.research.google.com/github/hsandaver/hsandaver/blob/main/ML_FADER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install --quiet opencv-python-headless ipywidgets Pillow scipy pandas scikit-learn

# Enable ipywidgets extension for Google Colab
!jupyter nbextension enable --py widgetsnbextension

# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import ipywidgets as widgets
from PIL import Image
from google.colab import files
import io  # For handling file input/output
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
import os  # For file operations

warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

# 1. Create More Accurate Synthetic Data with Environmental Effects
def create_more_accurate_synthetic_data(num_samples=1000):
    """
    Generates synthetic data simulating the fading of different materials under various environmental conditions.

    Parameters:
        num_samples (int): Number of samples to generate.

    Returns:
        pd.DataFrame: Synthetic dataset.
    """
    np.random.seed(42)

    # Define material-specific fading parameters based on research
    material_fading_types = {
        'Paper': {
            'uv_sensitivity': lambda: np.random.uniform(0.4, 1.0, num_samples),
            'fading_curve': lambda t: 50 * np.exp(-0.05 * t)  # Rapid initial fading followed by slow fading
        },
        'Textiles': {
            'uv_sensitivity': lambda: np.random.uniform(0.3, 0.8, num_samples),
            'fading_curve': lambda t: 30 * np.log1p(t)  # Gradual fading after a threshold
        },
        'Albumen Prints': {
            'uv_sensitivity': lambda: np.random.uniform(0.2, 0.7, num_samples),
            'fading_curve': lambda t: 10 + 0.3 * t  # Sensitive to UV, with pronounced yellowing
        },
        'Silver Gelatin Photographs': {
            'uv_sensitivity': lambda: np.random.uniform(0.1, 0.5, num_samples),
            'humidity_sensitivity': lambda: np.random.uniform(0.1, 0.6, num_samples),  # Humidity increases yellowing
            'fading_curve': lambda t: -20 * np.exp(-0.02 * t) + 30  # Initial darkening, then fading
        }
    }

    # Randomly assign materials to the samples
    materials = np.random.choice(list(material_fading_types.keys()), num_samples)

    # Initialize dataframe
    data = pd.DataFrame({
        'material': materials,
        'uv_exposure': [material_fading_types[m]['uv_sensitivity']()[i] for i, m in enumerate(materials)],
        'lux_hours': np.random.uniform(0, 500, num_samples),
        'humidity': np.random.uniform(0, 100, num_samples),
        'temperature': np.random.uniform(-10, 50, num_samples),
        'manufacture_year': np.random.randint(1600, 2025, num_samples),
        'time_years': np.random.uniform(0, 100, num_samples)
    })

    # Apply material-specific fading curves to generate color shifts
    def compute_delta_L(row):
        material = row['material']
        t = row['time_years']
        fading_curve = material_fading_types[material]['fading_curve'](t)
        if material == 'Silver Gelatin Photographs':
            # Incorporate humidity sensitivity for silver gelatin
            return fading_curve * row['uv_exposure'] * row['humidity'] * material_fading_types[material]['humidity_sensitivity']()[0]
        else:
            return fading_curve * row['uv_exposure']

    data['delta_L'] = data.apply(compute_delta_L, axis=1)
    data['delta_A'] = np.random.uniform(-50, 50, num_samples) * 0.1  # Smaller variation
    data['delta_B'] = np.random.uniform(-50, 50, num_samples) * 0.1  # Smaller variation

    # Clip delta values to realistic ranges
    data['delta_L'] = data['delta_L'].clip(-100, 100)
    data['delta_A'] = data['delta_A'].clip(-100, 100)
    data['delta_B'] = data['delta_B'].clip(-100, 100)

    return data

# Generate synthetic data
lab_data = create_more_accurate_synthetic_data()

# Display the first few rows of the data
print("Sample of Synthetic Data:")
display(lab_data.head())

# 2. Preprocess the Synthetic Data
def preprocess_data(data):
    """
    Preprocesses the synthetic data by encoding categorical variables, scaling features, and splitting into train and validation sets.

    Parameters:
        data (pd.DataFrame): The synthetic dataset.

    Returns:
        tuple: Scaled training and validation features and targets, scaler object, and the regression model.
    """
    # Drop any potential NaN values
    data = data.dropna()

    # Encode categorical variables using one-hot encoding
    data_encoded = pd.get_dummies(data, columns=['material'], drop_first=True)

    # Features and targets
    feature_cols = ['uv_exposure', 'lux_hours', 'humidity', 'temperature', 'manufacture_year', 'time_years']
    feature_cols += [col for col in data_encoded.columns if col.startswith('material_')]
    features = data_encoded[feature_cols]
    targets = data_encoded[['delta_L', 'delta_A', 'delta_B']]

    # Feature scaling
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(features)

    # Split the data
    X_train, X_val, y_train, y_val = train_test_split(X_scaled, targets, test_size=0.2, random_state=42)

    return X_train, X_val, y_train, y_val, scaler, feature_cols

# Preprocess data
X_train, X_val, y_train, y_val, scaler, feature_cols = preprocess_data(lab_data)

# 3. Train Multi-output Regression Model
def train_model(X_train, y_train, model_type='LinearRegression'):
    """
    Trains a multi-output regression model.

    Parameters:
        X_train (np.array): Training features.
        y_train (pd.DataFrame): Training targets.
        model_type (str): Type of regression model ('LinearRegression' or 'RandomForest').

    Returns:
        MultiOutputRegressor: Trained regression model.
    """
    if model_type == 'LinearRegression':
        base_model = LinearRegression()
    elif model_type == 'RandomForest':
        base_model = RandomForestRegressor(n_estimators=100, random_state=42)
    else:
        raise ValueError("Unsupported model type. Choose 'LinearRegression' or 'RandomForest'.")

    model = MultiOutputRegressor(base_model)
    model.fit(X_train, y_train)
    return model

# Choose model type
model_type = 'RandomForest'  # Options: 'LinearRegression', 'RandomForest'

# Train the model
model = train_model(X_train, y_train, model_type=model_type)

# 4. Evaluate the Model
def evaluate_model(model, X_val, y_val):
    """
    Evaluates the regression model using RMSE.

    Parameters:
        model (MultiOutputRegressor): The trained regression model.
        X_val (np.array): Validation features.
        y_val (pd.DataFrame): Validation targets.

    Returns:
        None
    """
    predictions = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, predictions, multioutput='raw_values'))
    print(f"Calibration Model RMSE (Validation Data):")
    print(f" - L channel: {rmse[0]:.2f}")
    print(f" - A channel: {rmse[1]:.2f}")
    print(f" - B channel: {rmse[2]:.2f}")

print("\nModel Evaluation:")
evaluate_model(model, X_val, y_val)

# 5. Interactive Interface for Fading Simulation
# Define material-specific environmental factor ranges
material_env_ranges = {
    'Paper': {
        'uv_exposure': (0.4, 1.0),
        'lux_hours': (100, 500),
        'humidity': (20, 80),
        'temperature': (10, 40),
        'time_years': (0, 50)
    },
    'Textiles': {
        'uv_exposure': (0.3, 0.8),
        'lux_hours': (150, 450),
        'humidity': (30, 90),
        'temperature': (15, 35),
        'time_years': (0, 70)
    },
    'Albumen Prints': {
        'uv_exposure': (0.2, 0.7),
        'lux_hours': (80, 400),
        'humidity': (25, 85),
        'temperature': (5, 45),
        'time_years': (0, 90)
    },
    'Silver Gelatin Photographs': {
        'uv_exposure': (0.1, 0.5),
        'lux_hours': (50, 300),
        'humidity': (10, 70),
        'temperature': (0, 50),
        'time_years': (0, 100)
    }
}

# Initialize widgets
material_dropdown = widgets.Dropdown(
    options=list(material_env_ranges.keys()),
    value='Paper',
    description='Material:',
    style={'description_width': 'initial'}
)

uv_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01, description='UV Exposure',
    style={'description_width': 'initial'}
)
lux_slider = widgets.FloatSlider(
    value=250, min=0, max=500, step=10, description='Lux Hours',
    style={'description_width': 'initial'}
)
humidity_slider = widgets.FloatSlider(
    value=50, min=0, max=100, step=1, description='Humidity (%)',
    style={'description_width': 'initial'}
)
temp_slider = widgets.FloatSlider(
    value=20, min=-10, max=50, step=1, description='Temperature (°C)',
    style={'description_width': 'initial'}
)
time_slider = widgets.FloatSlider(
    value=10, min=0, max=100, step=1, description='Years of Aging',
    style={'description_width': 'initial'}
)
manufacture_year_slider = widgets.IntSlider(
    value=1850, min=1600, max=2024, step=1, description='Year of Manufacture',
    style={'description_width': 'initial'}
)

# Function to update slider ranges based on selected material
def update_sliders(change):
    material = change['new']
    ranges = material_env_ranges[material]
    uv_slider.min, uv_slider.max = ranges['uv_exposure']
    uv_slider.value = np.clip(uv_slider.value, ranges['uv_exposure'][0], ranges['uv_exposure'][1])

    lux_slider.min, lux_slider.max = ranges['lux_hours']
    lux_slider.value = np.clip(lux_slider.value, ranges['lux_hours'][0], ranges['lux_hours'][1])

    humidity_slider.min, humidity_slider.max = ranges['humidity']
    humidity_slider.value = np.clip(humidity_slider.value, ranges['humidity'][0], ranges['humidity'][1])

    temp_slider.min, temp_slider.max = ranges['temperature']
    temp_slider.value = np.clip(temp_slider.value, ranges['temperature'][0], ranges['temperature'][1])

    time_slider.min, time_slider.max = ranges['time_years']
    time_slider.value = np.clip(time_slider.value, ranges['time_years'][0], ranges['time_years'][1])

# Attach the update function to material dropdown
material_dropdown.observe(update_sliders, names='value')

# Display sliders in a VBox
slider_box = widgets.VBox([
    material_dropdown,
    uv_slider, lux_slider, humidity_slider,
    temp_slider, time_slider, manufacture_year_slider
])
display(slider_box)

# 6. Upload and Process Image
def upload_image():
    """
    Allows the user to upload an image and converts it to the LAB color space.

    Returns:
        tuple: Original image in RGB and LAB color spaces.
    """
    try:
        uploaded = files.upload()
        if not uploaded:
            print("No file uploaded.")
            return None, None
        for filename in uploaded.keys():
            image = Image.open(io.BytesIO(uploaded[filename])).convert('RGB')
            display(image)
            image_np = np.array(image)
            image_bgr = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)
            image_lab = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2LAB)
            return image_np, image_lab
    except Exception as e:
        print(f"An error occurred while uploading the image: {e}")
        return None, None

# Call the function to upload an image
print("\nPlease upload an image to apply fading:")
original_image_rgb, original_image_lab = upload_image()

# 7. Define the Calibrated Fading Function
def fade_image(model, scaler, feature_cols, material, uv_exposure, lux_hours, humidity, temperature, manufacture_year, time_years):
    """
    Applies the calibrated fading effect to the uploaded image based on environmental factors.

    Parameters:
        model (MultiOutputRegressor): Trained regression model.
        scaler (StandardScaler): Scaler used for feature scaling.
        feature_cols (list): List of feature column names.
        material (str): Selected material type.
        uv_exposure (float): UV exposure value.
        lux_hours (float): Lux hours value.
        humidity (float): Humidity percentage.
        temperature (float): Temperature in °C.
        manufacture_year (int): Year of manufacture.
        time_years (float): Years of aging.

    Returns:
        np.array: Faded image in LAB color space.
    """
    # Prepare input features
    input_features = {
        'uv_exposure': uv_exposure,
        'lux_hours': lux_hours,
        'humidity': humidity,
        'temperature': temperature,
        'manufacture_year': manufacture_year,
        'time_years': time_years
    }

    # One-hot encode material
    for mat in material_env_ranges.keys():
        if mat != material:
            input_features[f'material_{mat}'] = 0
        else:
            input_features[f'material_{mat}'] = 1

    # Create DataFrame in the order of feature_cols
    input_df = pd.DataFrame([input_features])[feature_cols]

    # Scale features
    input_scaled = scaler.transform(input_df)

    # Predict color shifts
    delta = model.predict(input_scaled)[0]

    # Apply color shifts
    faded_lab = original_image_lab.copy().astype(np.float32)
    faded_lab[:, :, 0] = np.clip(faded_lab[:, :, 0] - delta[0], 0, 255)  # L channel
    faded_lab[:, :, 1] = np.clip(faded_lab[:, :, 1] + delta[1], 0, 255)  # A channel
    faded_lab[:, :, 2] = np.clip(faded_lab[:, :, 2] + delta[2], 0, 255)  # B channel

    return faded_lab.astype(np.uint8)

# 8. Set Up Fading Function with Color Mapping and Download Option
output = widgets.Output()

def on_apply_fading_clicked(b):
    """
    Event handler for the "Apply Fading" button. Applies the fading effect and displays the result.
    """
    with output:
        clear_output()
        if original_image_lab is None:
            print("Please upload an image first.")
            return

        material = material_dropdown.value
        uv_exposure = uv_slider.value
        lux_hours = lux_slider.value
        humidity = humidity_slider.value
        temperature = temp_slider.value
        manufacture_year = manufacture_year_slider.value
        time_years = time_slider.value

        try:
            faded_image_lab = fade_image(
                model, scaler, feature_cols, material,
                uv_exposure, lux_hours, humidity,
                temperature, manufacture_year, time_years
            )

            # Convert faded LAB image back to RGB
            faded_image_bgr = cv2.cvtColor(faded_image_lab, cv2.COLOR_LAB2BGR)
            faded_image_rgb = cv2.cvtColor(faded_image_bgr, cv2.COLOR_BGR2RGB)

            # Display original and faded images side by side
            fig, ax = plt.subplots(1, 2, figsize=(12, 6))
            ax[0].imshow(original_image_rgb)
            ax[0].set_title("Original Image")
            ax[0].axis('off')
            ax[1].imshow(faded_image_rgb)
            ax[1].set_title(f"Faded Image ({time_years} Years)")
            ax[1].axis('off')
            plt.show()

            # Provide option to download the faded image
            faded_pil = Image.fromarray(faded_image_rgb)
            buf = io.BytesIO()
            faded_pil.save(buf, format='PNG')
            buf.seek(0)
            # Define a unique filename for each download
            filename = f"faded_image_{int(time_years)}_years.png"
            with open(filename, 'wb') as f:
                f.write(buf.getvalue())
            download_button = widgets.Button(
                description="Download Faded Image",
                button_style='info',
                tooltip='Download the faded image as PNG',
                icon='download'
            )

            def on_download_clicked(b):
                try:
                    files.download(filename)
                except FileNotFoundError:
                    print("File not found. Please try applying the fading effect again.")

            download_button.on_click(on_download_clicked)
            display(download_button)

        except Exception as e:
            print(f"Error during fading: {e}")

apply_button = widgets.Button(
    description="Apply Fading",
    button_style='success',
    tooltip='Click to apply fading effect',
    icon='paint-brush'
)
apply_button.on_click(on_apply_fading_clicked)
display(apply_button, output)